In [ ]:
import os
import glob2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mplstereonet               #stereonet package 
import ternary as te              #python-ternary diagram package
from numpy.linalg import eigh
from matplotlib.backends.backend_pdf import PdfPages #This is to generate a multipage pdf for all the figures
import requests
import io

# 1.0 Directory and file list

In [ ]:
#Use this section if you are importing several csv files locally (from your computer)

#For online repository downloads, go directly to section 2.1 below

# os.chdir(r'C:\path_to_file') #You can use this to change the directory

In [ ]:
# pwd #This is to show the path to the files

In [ ]:
# os.listdir() #This will show the list of files

In [ ]:
# filelist=glob2.glob('*.csv') #To create a list of all the csv files

In [ ]:
#This is to create an index of the file list

# for index, file in enumerate(filelist):
    # print(index, file)


In [ ]:
# value=filelist[2]   #This is to show that a list can be sliced by index
# print(value)

In [ ]:
# print(value[:-4]) #This shows that the string of filename can also be sliced. This will be useful for labels on figures

# 2.0 Import and read the data

In [ ]:
#Import a csv file from the file list

#index=int(input("What is the index nb of the file?"))  

#This ensures that the index value is treated as an integer (not a string)




In [ ]:
#df=[pd.read_csv(file) for file in filelist][index]   #This will read the chosen csv file and create a Pandas dataframe

In [ ]:
#df

## 2.1 Download and read the data from an online repository

In [ ]:
# Download a csv file from a public GitHub account

url = "https://raw.githubusercontent.com/GlacialGeo/PythonDemos/main/DATA/Fabric_Demo.csv" # Make sure the url is the raw version of the file on GitHub
download = requests.get(url).content

# Read the downloaded content and create a pandas dataframe

df = pd.read_csv(io.StringIO(download.decode('utf-8')))

# Print out the first 5 rows of the dataframe

df.head()

# 3.0 Data projections

In [ ]:
#Let's prepare the data to create stereonets and rose diagrams

#Create an array from the 'trend' column of the dataframe and display horizontally (one line)
Trend=df.loc[:,'Trend']
Trend=np.hstack(Trend)

#Do the same for the plunge data

Plunge=df.loc[:,'Plunge']
Plunge=np.hstack(Plunge)
print(Trend)

In [ ]:
Trend.shape

In [ ]:
#This is to prepare the data to build a rose diagram

#Calculate the number of directions (bins) every 10° using numpy.histogram.

bin_edges = np.arange(-5, 366, 10)                #numpy.arange(start, stop, step)
trends_in_bins, bin_edges = np.histogram(Trend, bin_edges)

In [ ]:
Trends=trends_in_bins[0:-1]
print(Trends)

In [ ]:
#Initialize the pdf file that will contain all the figures
#The name of the pdf file will be the same as the csv file name

#pp = PdfPages(f'Figures_{filelist[index][:-4]}.pdf')
#pp = PdfPages('FabricDemo.pdf')

#Create the rose diagram and the stereonets.

fig = plt.figure(figsize=(12, 12))         #creates an empty figure with no Axes

#Stereonet
ax1 = fig.add_subplot(221, projection='stereonet')
ax1.line(Plunge, Trend, 'o', color='blue')
ax1.set_title(f'a-axis on stereonet - Fabric Demo', y=1.10, fontsize=15)

#Rose diagram
ax2 = fig.add_subplot(222, projection='polar')
ax2.set_title('a-axis on rose diagram', y=1.10, fontsize=15)

ax2.bar(np.deg2rad(np.arange(0, 360, 10)), Trends, 
       width=np.deg2rad(10), bottom=0.0, color='.8', edgecolor='k')
ax2.set_theta_zero_location('N')
ax2.set_theta_direction(-1)
ax2.set_thetagrids(np.arange(0, 360, 30), labels=np.arange(0, 360, 30))
ax2.set_rgrids(np.arange(1, Trends.max() + 4, 3), angle=0, weight= 'black')

#Stereonet with kamb density
ax3 = fig.add_subplot(223, projection='stereonet')
ax3.line(Plunge, Trend, 'o', color='blue')
m=ax3.density_contourf(Plunge, Trend, measurement='lines', cmap='Reds') #exponential_kamb is the default method
ax3.set_title('with kamb density contours', y=1.05, fontsize=15)
fig.colorbar(m) 


#Specify a few things for some of the plots
for ax in [ax1, ax3]:
    ax.grid()
    ax.set_azimuth_ticks([]) #This is to hide the azimuth labels (optional)

for ax in [ax1]:
    note = f"n={Plunge.size} \nMean Plunge: {Plunge.mean():.1f}"
    ax.annotate(note, xy=(5*60, -30), xycoords='axes points')  

plt.show()
fig.savefig('Fabric_Demo.png', bbox_inches = 'tight', format='png')
#pp.savefig(fig)


## 4.0 Fabric Analysis

In [ ]:
#Get the eigenvalues and eigenvectors using mplstereonet

plu, azi, vals = mplstereonet.eigenvectors(Plunge, Trend, measurement='lines') 
#This returns 1-D arrays for plunge and azimuth (eigenvectors converted to spherical coordinates), and normalized eigenvalues
print(plu,azi,vals)

In [ ]:
#Extract the normalized eigenvalues
S1=vals[0].round(4)
S2=vals[1].round(4)
S3=vals[2].round(4)
print(S1,S2,S3)
#The eigenvalues are slightly different with mplstereonet than with other tools/softwares (e.g. Stereonet 11)

In [ ]:
#So, let's try a different method to get those values

#Calculate direction cosines...
def get_dir_cosines(Trend, Plunge):
    a=np.cos(Trend*np.pi/180)
    b=np.cos(Plunge*np.pi/180)     
    c=np.sin(Trend*np.pi/180)
    x=(a*b).round(4) #first direction cosines.
    y=(c*b).round(4) #second direction cosines
    z=np.sin(Plunge*np.pi/180).round(4) #third direction cosines
    return x, y, z

#To bring the three direction cosines (x, y, z) together in a single (N,3) array (N=nb# of measurements):
X=np.column_stack(get_dir_cosines(Trend, Plunge))
print(X)

In [ ]:
X.shape

In [ ]:
XT=X.T      #Transpose matrix X (X')
print(XT)

In [ ]:
XT.shape

In [ ]:
A=np.dot(XT,X).round(4) #This produces a 3X3 matrix (X'X) of the sums of squares and cross products (SSCP) of the direction cosines
print(A)                   

In [ ]:
#Using numpy...

Ei=eigh(A)  #This returns a 1-D array (eigenvalues) and a 2-D array (eigenvectors)
print(Ei)

In [ ]:
#Extract the eigenvalues

Eighv=Ei[0] 
print(Eighv)

In [ ]:
#The sum of eigenvalues equals to the nb# of measurements. 
#Normalized eigenvalues (divided by number of measurements) sum to 1.

S1=Eighv[2]/Trend.shape
S2=Eighv[1]/Trend.shape
S3=Eighv[0]/Trend.shape
print(S1, S2, S3)  
#normalized eigenvalues; the values obtained with 'eigh' are the same as those obtained using Stereonet 11

In [ ]:
#Make a biplot of S3 and S1

fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(1,1,1) # row-col-num

# Hide the right and top lines of the default box
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

#ax.set_xlim((0.4, 1.0,))
#ax.set_ylim((0.0, 0.3))

xticks = np.arange(0.4, 1.0, 0.1).round(2)
yticks = np.arange(0.0, 0.3, 0.05).round(2)

xtickLocations=np.arange(0.4, 1.0, 0.1).round(2)
yticklocations=np.arange(0.0, 0.3, 0.05).round(2)


ax.scatter(S1,S3, c='green', s=100)
ax.set_xlabel('S1', fontsize=18)
ax.set_ylabel('S3', fontsize=18)
ax.set_xticks(ticks=xtickLocations)
ax.set_yticks(ticks=yticklocations)
ax.set_xticklabels(xticks, fontsize=16)
ax.set_yticklabels(yticks, fontsize=16)

ax.grid(True, linestyle='--')
plt.show()
#fig.savefig(f'S3_S1plot_{filelist[index][:-4]}.png', bbox_inches = 'tight', format='png')
pp.savefig(fig)

In [ ]:
#Extract V1, V2, and V3

V1_azi=azi[0].round(1)
V1_plunge=plu[0].round(1)
V2_azi=azi[1].round(1)
V2_plunge=plu[1].round(1)
V3_azi=azi[2].round(1)
V3_plunge=plu[2].round(1)
print("V1 is", "N",V1_azi,"/",V1_plunge)
print("V2 is", "N",V2_azi,"/",V2_plunge)
print("V3 is", "N",V3_azi,"/",V3_plunge)

In [ ]:
E=(1-(S2/S1)).round(4)     #Elongation index

In [ ]:
I=(S3/S1).round(4)        #Isotropy index

In [ ]:
R=(1-(E+I)).round(4)    #A residual value to allow plotting the indices correctly on a ternary diagram

In [ ]:
df=pd.DataFrame(columns=['V1_trend', 'V1_plunge', 'E','I', 'R'])

In [ ]:
df.loc[0]=[V1_azi, V1_plunge, E,I,R]
df

In [ ]:
df["E"]=df['E'].astype('float')      #To specify the Dtype is 'float'
df["I"]=df['I'].astype('float')
df["R"]=df['R'].astype('float')
df.dtypes

In [ ]:
df.info()

In [ ]:
df

In [ ]:
#Export the Dataframe as a csv.

#df.to_csv(f'Data_output_{filelist[index][:-4]}.csv', index=False)

In [ ]:
#This function asks the user to classify the fabric modality and returns a value from 0 to 4 (to be used to plot modality)

List=['un', 'su', 'bi', 'sb', 'mm']

def modality():
    print(List)
    Mod=str(input("What is the modality of the fabric? "))
    if Mod=='un':
        print("Thank you")
        return 0
    elif Mod=='su':
        print("Thank you")
        return 1
    elif Mod=='bi':
        print("Thank you")
        return 2
    elif Mod=='sb':
        print("Thank you")
        return 3
    elif Mod=='mm':
        print("Thank you")
        return 4
    else:
        print("This is not a valid modality")

Modal=modality()

In [ ]:
#This is to create the modality-isotropy plot

fig = plt.figure(figsize=(12.5,8))
ax = fig.add_subplot(1,1,1) # row-col-num

ylocations=[0,1,2,3,4]
labels=["un", 'su', 'bi', 'sb', 'mm']

# Hide the right and top lines of the default box
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

#PLot the dots and assign ticks and labels
ax.scatter(I, Modal, s=200)
ax.set_yticks(ticks=ylocations)
ax.set_yticklabels(labels, fontsize=18)

xticks = [ 0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6 ]
ax.set_xticks(xticks)
ax.set_xticklabels(xticks, fontsize=18)

plt.xlabel(r'S3/S1 (isotropy)', fontsize=16)
plt.ylabel('Modality', fontsize=18)

#Add the grid lines and show the plot
ax.grid(True, linestyle='--')
plt.show()
#fig.savefig(f'Modality_IsotropyPlot_{filelist[index][:-4]}.png', bbox_inches = 'tight', format='png')
pp.savefig(fig)

In [ ]:
#Let's plot the results on a ternary diagram!

scale = 1.0
figure, fabric = te.figure(scale=scale)
figure.set_size_inches(12,10)

#PLot the data
fabric.scatter(df[['E','I','R']].values, marker='D', color='green', label="Green Diamonds")
    
# Draw Boundary and Gridlines
fabric.boundary(linewidth=2.0)
fabric.gridlines(color="blue", multiple=0.2)
    
# Set Axis labels
fontsize = 12
offset = 0.2
fabric.left_axis_label("I=S3/S1", fontsize=fontsize, offset=0.2)
fabric.right_axis_label("E=1-(S2/S1)", fontsize=fontsize, offset=0.2)
fabric.top_corner_label("Isotropic", fontsize=fontsize, offset=0.25)
fabric.right_corner_label("Cluster", fontsize=fontsize, offset=-0.05)
fabric.left_corner_label("Girdle", fontsize=fontsize, offset=-0.05)

#This is to configure the style of the axes and ticks and specify their orientation/sense
fabric.ticks(axis='lbr', multiple=0.2, linewidth=1, offset=0.025, tick_formats="%.1f", clockwise=True)
fabric.get_axes().axis('off')
fabric.clear_matplotlib_ticks()

fabric.show()
#fabric.savefig(f'Ternary_Diagram_{filelist[index]}.svg', bbox_inches = 'tight', format='svg')
pp.savefig(figure)

In [ ]:
#Now V1 can be added to the data points on the stereonet

fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111, projection='stereonet')
ax.line(Plunge, Trend, 'o', color='blue', label='a-axis')
plunge=V1_plunge
bearing=V1_azi
ax.line(plunge, bearing, 'X', color='green', markersize=12, label="V1")
ax.set_title('a-axis on stereonet with V1', y=1.10, fontsize=15)
    
#An approach to avoid repetition of labels in the legend:
from collections import OrderedDict

handles, labels = ax.get_legend_handles_labels()
by_label = OrderedDict(zip(labels, handles))

#This is to display the legend in the upper right corner without overlapping the stereonet
ax.legend(by_label.values(), by_label.keys(), loc='upper right', bbox_to_anchor=(1.1, 1.1), fontsize=14)

#This is another approach to remove duplicate labels in the legend
#def legend_without_duplicate_labels(ax):
    #handles, labels = ax.get_legend_handles_labels()
    #unique = [(h, l) for i, (h, l) in enumerate(zip(handles, labels)) if l not in labels[:i]]
    #ax.legend(*zip(*unique))


#Add some notes and values beside the plot
note = f"n={Plunge.size} \nS1={S1} \nV1 azimuth={V1_azi}"
ax.annotate(note, xy=(5*60, -30), xycoords='axes points')


ax.grid()
ax._polar.set_position(ax.get_position()) #manually matches any size changes that have occurred in the parent axis
plt.show()
#fig.savefig(f'Stereonet_w_V1_{filelist[index]}.svg', bbox_inches = 'tight', format='svg')
pp.savefig(fig)

In [ ]:
pp.close()  #This is to close the pdf file

In [ ]:
#Output a proposed interpretation based on the above results and plots
#It will also save the interpretation output in a *.txt file 

with open(f"Interpretation_Demo.txt", "w") as external_file:
    if Modal<=2 and S1>=0.72 and I<0.12: #un or su and some bi
        file_contents = external_file.write('Strong till clast fabric; Can be used for paleo-ice flow')
    elif Modal>=2 and S1>=0.68 and S1<0.72 and I<=0.15:  #bi or sb
        file_contents = external_file.write('Moderately strong till clast fabric; use for paleo-ice flow with caution')
    elif Modal>=2 and S1>0.55 and S1<0.7 and I<0.2: #sb and maybe some bi
        file_contents = external_file.write('Moderate till clast fabric; interpret paleo-ice flow with great caution; check other data and local context')
    elif S1<=0.55 and I>=0.2: #mm
        file_contents = external_file.write('Weak till clast fabric; unreliable for paleo-ice flow')
    else:
        file_contents = external_file.write('Undefined; check data and plots')

In [ ]:
if V1_azi>=0 and V1_azi<180:
    Ice_Flow=(V1_azi+180).round()
else: 
    Ice_Flow=(V1_azi-180).round()
          

In [ ]:
#This will add the ice flow interpretation to the same *.txt file as above, but on a different line

with open(f"Interpretation_Demo.txt", "a+") as external_file:
    external_file.seek(0)          #This will go to the top/start of text
    data = external_file.read()
    if len(data) > 0 :
        external_file.write("\n")
    if S1>0.57 and V1_plunge>2:
        external_file.write(f'Paleo-ice flow was likely toward {Ice_Flow}')
    elif S1>0.57 and V1_plunge<2:
        external_file.write("Orientation likely reliable but V1 plunge too low; direction could be opposite; check local context")
    else: 
        external_file.write("Paleo-ice flow direction is uncertain/unreliable")

In [ ]:
with open(f"Interpretation_Demo.txt", "r") as external_file:
    print(external_file.read())

In [ ]:
#import the csv containing the combined outputs
#df2 = pd.read_csv(r'C:\Users\...\Data_output_Fabrics_all.csv')
#df2

#Let's plot all the results on a ternary diagram!

scale = 1
figure, fabric = te.figure(scale=scale)
figure.set_size_inches(12,10)

#PLot the data
fabric.scatter(df2[['E','I','R']].values, marker='o', c='blue', label='ALL')
    
# Draw Boundary and Gridlines
fabric.boundary(linewidth=2.0)
fabric.gridlines(color="blue", multiple=0.2)
    
# Set Axis labels
fontsize = 12
offset = 0.2
fabric.left_axis_label("I=S3/S1", fontsize=fontsize, offset=0.2)
fabric.right_axis_label("E=1-(S2/S1)", fontsize=fontsize, offset=0.2)
fabric.top_corner_label("Isotropic", fontsize=fontsize, offset=0.25)
fabric.right_corner_label("Cluster", fontsize=fontsize, offset=-0.05)
fabric.left_corner_label("Girdle", fontsize=fontsize, offset=-0.05)

#This is to configure the style of the axes and ticks and specify their orientation/sense, and add a legend
#fabric.legend(title="Fabric nb", fontsize=14)
fabric.ticks(axis='lbr', multiple=0.2, linewidth=1, offset=0.025, tick_formats="%.1f", clockwise=True)
fabric.get_axes().axis('off')
fabric.clear_matplotlib_ticks()

fabric.show()
fabric.savefig(f'Ternary_Diagram_all.png', bbox_inches = 'tight', format='png')
#pp.savefig(figure)